In [1]:
pip install scikit-surprise

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Berry\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import cross_validate

In [3]:
ratings_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/ratings.csv')
movies_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/movies.csv')
data = pd.merge(ratings_df, movies_df, on='movieId')
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(data[['userId','movieId','rating']], reader)

In [4]:
sim_options = {'name':'cosine',
               'user_based': False} # content based filtering
algo = KNNBasic(sim_options=sim_options)

In [5]:
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9728  0.9734  0.9778  0.9787  0.9691  0.9744  0.0035  
Fit time          6.53    6.75    6.72    6.85    6.80    6.73    0.11    
Test time         3.71    3.79    3.69    4.03    3.74    3.79    0.12    


{'test_rmse': array([0.97277533, 0.97338719, 0.9777835 , 0.97871886, 0.96912421]),
 'fit_time': (6.530308723449707,
  6.752989292144775,
  6.718516826629639,
  6.851621389389038,
  6.801561594009399),
 'test_time': (3.7076854705810547,
  3.7860145568847656,
  3.6926558017730713,
  4.032233238220215,
  3.738647222518921)}

In [6]:
algo.fit(data.build_full_trainset())
print(algo.sim)

Computing the cosine similarity matrix...
Done computing similarity matrix.
[[1.         0.90773108 0.96657344 ... 1.         1.         1.        ]
 [0.90773108 1.         0.88581521 ... 1.         1.         1.        ]
 [0.96657344 0.88581521 1.         ... 1.         1.         1.        ]
 ...
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]]


In [7]:
pred = algo.predict(uid=1, iid=10)
pred.est

3.9880309408033616

In [8]:
df = pd.merge(ratings_df, movies_df, on='movieId')

In [9]:
import seaborn as sns
import numpy as np


In [10]:
from surprise.model_selection import train_test_split
from surprise import accuracy

In [11]:
trainset, testset = train_test_split(data, test_size=0.25)

In [12]:
# Using the same initial setup as in the user-based example

# Using KNN to compute similarities between items
algo = KNNBasic(sim_options={'user_based': False})
algo.fit(trainset)

# Making predictions and evaluating the model
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9195


0.919495395031428

In [13]:
# SVD

from surprise import SVD

In [14]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8669  0.8693  0.8720  0.8638  0.8776  0.8699  0.0047  
Fit time          0.70    0.71    0.71    0.69    0.70    0.70    0.01    
Test time         0.14    0.06    0.14    0.06    0.14    0.11    0.04    


{'test_rmse': array([0.86686119, 0.86934864, 0.87197537, 0.86375523, 0.87764142]),
 'fit_time': (0.7014322280883789,
  0.7074480056762695,
  0.7077608108520508,
  0.6936295032501221,
  0.6968228816986084),
 'test_time': (0.13912653923034668,
  0.05504965782165527,
  0.14012718200683594,
  0.058053016662597656,
  0.1411290168762207)}

In [15]:
trainset2 = data.build_full_trainset()
svd.fit(trainset)

In [16]:
# predictions for everyone

df['predicted_rating'] = df.apply(lambda x: svd.predict(x['userId'], x['movieId']).est, axis=1)